In [0]:
from pyspark.sql import functions as F

In [0]:
source = dbutils.widgets.get("source")
table = dbutils.widgets.get("table")

In [0]:
df = spark.read.format("csv").option("header", True).load(f"abfss://raw@storageawesum.dfs.core.windows.net/staging/{source}/{table}/*/")

# Extract the full file path (input_file_name gives complete abfss path)
df = df.withColumn("file_path", F.input_file_name())
# Extract the folder name containing the timestamp
df = df.withColumn("extract_timestamp", F.regexp_extract("file_path", fr"{table}_(\d{{8}}_\d{{6}})", 1))
df = df.drop(F.col("file_path"))

display(df)